In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
from astropy.coordinates import SkyCoord
import pickle, json, sys, os
import pandas as pd
from sys import getsizeof
pd.set_option('display.max_columns', None)

# location of the scripts
sys.path.insert(0, os.path.join("/fefs/aswg/workspace/juan.jimenez/lst1_systematics/scripts"))
import auxiliar as aux
import documents as docs

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# =================================== #
# paths 
fname_dict_sed_lc = "objects/dict_sed_and_lc.pkl"
fname_dict_dl2    = "objects/dict_dl2.pkl"

fname_total_dict  =  "objects/dict_runs.pkl"
# =================================== #


# =================================== #
#         intensity profiles
# =================================== #

# intensity binning
binsI = np.linspace(1.1, 4, 200) 

# =================================== #

### Reading all the different dicts

In [2]:
%%time
# light curve data
with open(fname_dict_sed_lc, "rb") as f:
    dict_gammapy = pickle.load(f)
    
# load dl2 data
with open(fname_dict_dl2, "rb") as f:
    dict_dl2 = pickle.load(f)

CPU times: user 987 ms, sys: 23.3 s, total: 24.2 s
Wall time: 3min 1s


### Converting the lightcurve dict to a run-wise dict

In [3]:
dict_LC_run = {}
for i, run in enumerate(np.sort(np.array(dict_gammapy["lightcurve"]["run_number"]).astype(int))):
    
    tmp_dict = {
        
        "flux"      : dict_gammapy["lightcurve"]["flux"][i]   * 1e10,  # cm-2 s-1
        "e_flux"    : dict_gammapy["lightcurve"]["e_flux"][i] * 1e10,  # cm-2 s-1
        "timestamp" : dict_gammapy["lightcurve"]["t_start"][i],
        "duration"  : dict_gammapy["lightcurve"]["timedelta"][i]
        
    }
    
    dict_LC_run[run] = tmp_dict

### Adding the LC and SED information to a total dict

In [22]:
dict_total = {}

dict_total["general"] = {
    
    "target_name"   : dict_gammapy["lightcurve"]["global"]["target_name"],
    "e_min_lc"      : dict_gammapy["lightcurve"]["global"]["e_min"], # TeV
    "e_max_lc"      : dict_gammapy["lightcurve"]["global"]["e_max"], # TeV
    "n_off_regions" : dict_gammapy["lightcurve"]["global"]["n_off_regions"],
    "chi2_flux"     : dict_gammapy["lightcurve"]["global"]["chi2"],
    "pvalue_flux"   : dict_gammapy["lightcurve"]["global"]["pvalue"]
    
}

_ampl, _e_ampl = dict_gammapy["dict_model"]["spectral"]["parameters"][0]["value"], dict_gammapy["dict_model"]["spectral"]["parameters"][0]["error"]
_ampl_unit     = u.Unit(dict_gammapy["dict_model"]["spectral"]["parameters"][0]["unit"])

_ref      = dict_gammapy["dict_model"]["spectral"]["parameters"][1]["value"]
_ref_unit = u.Unit(dict_gammapy["dict_model"]["spectral"]["parameters"][1]["unit"])

_a, _e_a = dict_gammapy["dict_model"]["spectral"]["parameters"][2]["value"], dict_gammapy["dict_model"]["spectral"]["parameters"][2]["error"]
_b, _e_b = dict_gammapy["dict_model"]["spectral"]["parameters"][3]["value"], dict_gammapy["dict_model"]["spectral"]["parameters"][3]["error"]

dict_total["general"]["model_params"] = {
    
        "amplitude" : (_ampl, _e_ampl) * _ampl_unit,
        "reference" :  _ref * _ref_unit,
        "alpha"     : (_a, _e_a),
        "beta"      : (_b, _e_b)
    
}

dict_total["general"]["SED"] = dict_gammapy["table_sed"]


dict_total["run"] = dict_LC_run

### Adding the dl2 data, and the intensity profiles in form of histogram counts, wheighted by the effective time

In [23]:
%%time
for run in np.sort(list(dict_dl2.keys())):
    
    dict_total["run"][run]["zd"]   = np.mean(dict_dl2[run]["zd"])
    dict_total["run"][run]["e_zd"] = (np.max(dict_dl2[run]["zd"]) - np.min(dict_dl2[run]["zd"])) / 2
    dict_total["run"][run]["az"]   = np.mean(dict_dl2[run]["az"])
    dict_total["run"][run]["e_az"] = (np.max(dict_dl2[run]["az"]) - np.min(dict_dl2[run]["az"])) / 2
    dict_total["run"][run]["effective_time"] = dict_dl2[run]["effective_time"]
    
    counts, Iedges = np.histogram(np.log10(dict_dl2[run]["intensity"]), binsI)
    Icenters = (Iedges[1:] + Iedges[:-1]) / 2
    counts   = counts / dict_dl2[run]["effective_time"]
    
    dict_total["run"][run]["intensity_rates"] = counts
    dict_total["run"][run]["bin_edges"]       = Iedges
    dict_total["run"][run]["bin_centers"]     = Icenters

CPU times: user 58 s, sys: 3.09 s, total: 1min 1s
Wall time: 1min 1s


In [24]:
dict_total["run"]

{'general': {'target_name': 'crab',
  'e_min_lc': <Quantity 0.1 TeV>,
  'e_max_lc': <Quantity 50. TeV>,
  'n_off_regions': 1,
  'chi2_flux': 211.9758236573264,
  'pvalue_flux': 1.341644685021356e-07,
  'model_params': {'amplitude': <Quantity [3.44909756e-11, 3.65307944e-13] 1 / (cm2 s TeV)>,
   'reference': <Quantity 1. TeV>,
   'alpha': (2.449758278448402, 0.00990913513049369),
   'beta': (0.11960538931788733, 0.007130151982698374)},
  'SED': <Table length=14>
         e_ref               e_min               e_max        ...                stat_scan                        norm_scan        
          TeV                 TeV                 TeV         ...                                                                  
        float64             float64             float64       ...               float64[11]                      float64[11]       
  ------------------- ------------------- ------------------- ... ---------------------------------------- ------------------------
  0.12